In [ ]:
import pandas as pd
from arch.unitroot import ADF, KPSS
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Extrayendo variable a analizar desde base de datos de Yunque Capital
imae = "'MRD-SR-05-01'" #indice en base de datos de Yunque Capital
conn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                    server='<servername>', 
                    database='<databasename>', 
                    uid='<user>', 
                    pwd='<password>')
c= conn.cursor()
query = "EXEC selectVariable @INDICE =" + str(imae) #Query a la base de datos
df = pd.read_sql(query,conn) #Enviando query a base de datos
df["Fecha"]=pd.to_datetime(df['Fecha'], utc=True) #Cambiando formato de columna de fechas
df = df.set_index('Fecha') #Estableciendo indice de fechas
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
df.plot(title="IMAE", ax=ax)

In [ ]:
import seaborn as sns

#Separando mes y año para analizar estacionalidad
dfsns = df.copy()
dfsns["year"]=dfsns.index.year
dfsns["month"]=dfsns.index.strftime("%b")
#Creando grafico de parametros necesarios
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=dfsns,
            x="month",
            y="Valor",
            hue="year",
            style="year",
            legend="full",
            palette="colorblind")
plt.title("IMAE Original - Seasonal Plot")
plt.legend(bbox_to_anchor=(1.05,1),loc=2)

In [ ]:
#Test de estacionaridad de Dickey-Fuller
adf = ADF(df["Valor"])
print(adf.summary().as_text())

In [ ]:
def kpss_test(x, h0_type="c"): 
    indices = ["Test Statistic", "p-value", "# of Lags"]
    kpss_test = kpss(x, regression=h0_type)
    results = pd.Series(kpss_test[0:3], index=indices)
    
    for key, value in kpss_test[3].items():
        results[f"Critical Value ({key})"] = value
    return results

In [ ]:
kpss_test(df["Valor"])

In [ ]:
#Test de estacionaridad de Phillips-Perron
adf = KPSS(df["primera_diferencia"].dropna())
print(adf.summary().as_text())

In [ ]:
#Calcualndo diferencias
df["primera_diferencia"] = df.Valor.diff(1)

In [ ]:
#mpliar tamaño de grafico
fig, ax = plt.subplots(figsize=(10, 6))
#graficar primera diferencia
df["primera_diferencia"].plot(title="primera diferencia", ax=ax)

In [ ]:
#Test de estacionaridad de Dickey-Fuller
adf = ADF(df["primera_diferencia"].dropna())
print(adf.summary().as_text())

In [ ]:
import numpy as np
df["valor_log"] = np.log(df.Valor)

In [ ]:
df["valor_log"].plot()

In [ ]:
df["Valor"].pct_change().plot()

In [ ]:
df["primera_diferencia"]